In [2]:
import torch
from sys import stdout
from model2 import network, vocab, char2idx, idx2char, input_dim, hidden_dim, num_layers

prompt = "The next big invention "
speech_len = 1000
temperature = 0.55

if __name__ == "__main__":
    # Load model
    speaker = network(input_dim, hidden_dim, num_layers)
    speaker.load_state_dict(torch.load('./checkpoints/e2_1.2322.pth', map_location='cpu'))
    speaker.batch_size = 1
    speaker.reset()
    speaker.eval()

    output = prompt[-1]  # Start generation from last prompt letter

    with torch.no_grad():
        # Prime the model on the prompt (except last letter)
        for letter in prompt[:-1]:
            ip = torch.tensor([[char2idx[letter]]])
            _ = speaker(ip)

        stdout.write(prompt)

        # Generate text
        for _ in range(speech_len):
            ip = torch.tensor([[char2idx[output]]])  # Feed in last predicted letter
            logits = speaker(ip).squeeze()           # Output logits
            probs = torch.softmax(logits / temperature, dim=0)  # Apply temperature
            top_i = torch.multinomial(probs, 1).item()          # Sample from distribution
            output = idx2char[top_i]
            stdout.write(output)


The next big invention VgHVH3h&K3,'H3GKY3T
fY3,GYglg(,YgG@53,G_3Y&H3hK'V_3gf3Y&H3V,G_53^K
3l,G3sH3,sVH3YK3sH3fKrH3K%3Y&H3fY'HHY3,G_3Y&H3V,fY3YgrH3#3hK
V_3f,^3Y&,Y3hH3GHH_3YK3Hx(VK'H3Y&Hr53,G_3h&,Y3hH3&,_3,3GHh3(,'Y3K%3fKrHY&gG@3Y&,Y3r,^sH3#3h,f3gG3Y&H3YHl&GKVK@gHf3fg_H3K%3Y&H3f(HlY'
lYgKG3K%3,VV3Y&H3fYK'^3gG3&gf3*
HfYgKG3Y&,Y3#3hK
V_3&,CH3,G3Hx,r(VH3K%3Y&H3lK
'fH3K%3Y&H3rKfY3('K%gVH3YH,r3rKY&H'3KG3Y&H3K((K'Y
GgY^3YK3('K_
lH3fY'
lY
'H3YK3sH3,sVH3YK3_K3h&,Y3Y&H^3,'H3,Vh,^f3%gG_gG@3K
Y3h&HG3#3h,f3hK'DgG@3,G_3Y&H3(KK'3K%3Y&H3rKfY3(HK(VH3_gH_3YK3Y&H3s,lYH'g,3Y&,Y3hH3&,CH3YK3&,CH3,3VKY3K%3^H,'f3,@K53,G_3fK3Y&H^3@K3KG3YK3r^3@',GYH_3YK3fHCHG3K
Y3h&,Y3#3&,CH3sHHG3fHGfH3K
Y3K%3Y&H3f
sTHlY3K%3Y&H3l,fH53,G_3Y&H^d'H3GKY3gGYH'HfYgG@3gG3K
'3fKV,'k37&,Y3('HfHGY3#3h,GY3YK3&,CH3,3lKr(
YH'k3#3&,_3,3@'H,Y3f
sY3fg@Gg%gl,GY3flgHGlH3,G_3Y&H3r,gG3%'Kr3&gf3&,'_3YK3Y&H3('KsVHr3gG3Y&H3H,'Y&k3cHYdf3l,VV3Y&H3&
r,G3fY'
lY
'H3K%3Y&H3f,rH3YgrH3,G_3Y&H3('KsVHr3Y&,Y3hH3&,CH3Y&H3hK'V_53Y&H^3,'H3fK3r
l&3rK'H3Y&,G3Y&H3fgY